In [1]:
import numpy as np
np.random.seed(10)

In [2]:
import tensorflow as tf

C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [3]:
from PIL import Image

In [ ]:
#抓取圖片
import catch_photo as cp
def dogs_photo(local_path,dog):
    cp.got_photo(local_path,dog)


In [5]:
import glob
#儲存圖片陣列
files = []
#抓取c0,c1,c2 folder的圖片
for i in range(0,3):
    files.append(glob.glob ('C:\pythonwork\photo\c'+str(i)+'\*.jpg'))
    
len(files[0])

1000

In [7]:
#訓練陣列
x_train=[]
#真實陣列
y_train=[]
def standardization(image):
    #處理圖片
    image = Image.open(image)
    #L = 灰階
    image = image.convert("L")
    #寬30，長30
    new_width  = 30
    new_height = 30
    image = image.resize((new_width, new_height), Image.ANTIALIAS)
    return image
count = 0
for i in range(0,3):
    for myFile in files[i]:
        count = count + 1
        image = standardization(myFile)
        x_train.append (image)
        y_train.append (i)

In [8]:
print(len(x_train))
print(len(y_train))
x_train[0]

2827
2827


In [7]:
#圖片轉四維
print(x_train[0])
a=np.array(x_train[0])
print(a.shape)
print(a.ndim)
n=[]
n.append(a)
print(len(n))
data=np.array(n)
data.shape

<PIL.Image.Image image mode=L size=30x30 at 0x259D41A47F0>
(30, 30)
2
1


(1, 30, 30)

In [8]:
#圖片轉4維
nd3d_list = []
for i in range(0, len(x_train)):
    nd3d_list.append(np.array(x_train[i]))
x_data = np.array(nd3d_list)
#(圖片數,寬,高,色彩維度)
x_data=x_data.reshape(len(x_data),30,30,1)

print("shape of x_data : ", x_data.shape)

shape of x_data :  (2827, 30, 30, 1)


In [9]:
nd3d_list_1 = []
for i in range(0, len(y_train)):
    nd3d_list_1.append(np.array(y_train[i]))
y_data = np.array(nd3d_list_1)

print("shape of y_data : ", y_data.shape)
y_data[0]
y_data.shape

shape of y_data :  (2827,)


(2827,)

In [11]:
from keras.utils import np_utils
#分三類
num_classes=3
#標準化
x_data=x_data.astype('float32')/255
#載入真實數據
y_data=np_utils.to_categorical(y_data,num_classes)

In [12]:
#使用keras模組
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D

In [13]:
#線性堆疊模組
model=Sequential()
#卷積層1 設定激活函數
model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(30,30,1),activation='relu',padding='same'))
#放棄25%神經元以免overfitting
model.add(Dropout(0.25))

In [14]:
#池化層1
model.add(MaxPooling2D(2,2))

In [15]:
#卷積層2
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Dropout(0.25))

In [16]:
#池化層2
model.add(MaxPooling2D(2,2))

In [17]:
#平坦層
model.add(Flatten())
model.add(Dropout(0.25))

In [18]:
#隱藏層
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.25))

In [19]:
#輸出層
model.add(Dense(num_classes,activation='softmax'))

In [20]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        320       
_________________________________________________________________
dropout (Dropout)            (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0

In [21]:
#設定訓練
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [22]:
#開始訓練
#訓練資料占八成，週期為10，批次為2
train_history=model.fit(x_data,y_data,validation_split=0.2,epochs=10,batch_size=128,verbose=2)

W1230 15:26:45.429426 65868 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2261 samples, validate on 566 samples
Epoch 1/10
2261/2261 - 8s - loss: 1.0507 - accuracy: 0.5170 - val_loss: 1.9448 - val_accuracy: 0.0000e+00
Epoch 2/10
2261/2261 - 5s - loss: 0.8079 - accuracy: 0.6829 - val_loss: 1.6878 - val_accuracy: 0.0177
Epoch 3/10
2261/2261 - 5s - loss: 0.7185 - accuracy: 0.7218 - val_loss: 1.5461 - val_accuracy: 0.0972
Epoch 4/10
2261/2261 - 5s - loss: 0.6709 - accuracy: 0.7439 - val_loss: 1.1499 - val_accuracy: 0.4187
Epoch 5/10
2261/2261 - 5s - loss: 0.6522 - accuracy: 0.7621 - val_loss: 1.4794 - val_accuracy: 0.3004
Epoch 6/10
2261/2261 - 5s - loss: 0.6265 - accuracy: 0.7567 - val_loss: 1.5993 - val_accuracy: 0.2527
Epoch 7/10
2261/2261 - 5s - loss: 0.6111 - accuracy: 0.7665 - val_loss: 1.5166 - val_accuracy: 0.3127
Epoch 8/10
2261/2261 - 5s - loss: 0.5902 - accuracy: 0.7762 - val_loss: 1.2122 - val_accuracy: 0.4541
Epoch 9/10
2261/2261 - 5s - loss: 0.5985 - accuracy: 0.7687 - val_loss: 1.6070 - val_accuracy: 0.3180
Epoch 10/10
2261/2261 - 5s - lo

In [23]:
#儲存模型
model.save("dogs_model_1000.h5")
print("saved model to disk")

saved model to disk


In [24]:
label_dict={0:"黃金獵犬",1:"德國",2:"哈士奇"}

In [9]:
from PIL import Image
img = standardization("img/2.JPEG")
img

In [26]:
#將圖片轉陣列
import numpy as np
sample=[]
sample.append(img)
a=np.array(sample[0])
print(a.shape)
a=a.reshape(1,30,30,1).astype("float32")/255
a.shape

(30, 30)


(1, 30, 30, 1)

In [27]:
import tensorflow.keras as keras
#載入模型
model = keras.models.load_model("dogs_model2.h5")

In [28]:
#a = 預測圖片
prediction=model.predict_classes(a)

In [29]:
print(label_dict[prediction[0]])
img

哈士奇
